# Import relevant libraries/modules etc

In [ ]:
from tqdm import tqdm

In [ ]:
import numpy as np
import xarray as xr
import rasterio
%matplotlib inline
from matplotlib.pyplot import *
from glob import glob
import os
import datetime
import pandas as pd
from rasterio_to_xarray import rasterio_to_xarray

# Create an array with MAIAC data

- create a function which reads in the files, determines the no data values, and creates a third dim 'time'
- create a list with all of the files to be put in the 3d array
- run the function on every element of list (map)

x = rasterio_to_xarray(r'D:\Annies_Dissertation\MAIAC_Test\Projected\2014\MAIACAAOT.h00v01.20140011255.hdf_proj.tif')

In [ ]:
def maiac_file_to_da(filename):
    da = rasterio_to_xarray(filename)
    
    da.values[da.values == -28672] = np.nan
    da.values[da.values == 0] = np.nan
    
    time_str = os.path.basename(filename)[17:-13]
    time_obj = datetime.datetime.strptime(time_str, '%Y%j%H%M')
    da.coords['time'] = time_obj
    
    return da

In [ ]:
files = glob(r'D:\Annies_Dissertation\MAIAC_Test\Projected\2014\*_proj.tif')

maiac_file_to_da(files[0])

In [ ]:
list_of_das = map(maiac_file_to_da, files)

In [ ]:
res = xr.concat(list_of_das, 'time')

- view the dataset
- view the different 3D timestamps

In [ ]:
res

In [ ]:
res.time

# Group data
- merge multiple images per day using the maximum AOT estimates
- the third dimension is now called date (rather than time)

In [ ]:
r = res.groupby('time.date').max(dim='time')

In [ ]:
r.date

# Analysis

In [ ]:
overall_mean = r.mean(dim='date')

In [ ]:
figure(figsize=(20, 12))
overall_mean.plot(robust=True)

In [ ]:
r['date'] = pd.to_datetime(r['date'])

In [ ]:
r['date']

In [ ]:
seasonal = r.groupby('date.month').max(dim='date')

In [ ]:
seasonal.coords

In [ ]:
figure(figsize=(20, 12))
seasonal.plot(col='month', robust=True)

# Time series over one pixel
- get the information (time series) over one pixel
- validate against AERONET Chibolton site

In [ ]:
df = r.isel(x=1103, y=1026).dropna('date').to_pandas()

In [ ]:
df

In [ ]:
df.to_csv(r'D:\Annies_Dissertation\MAIAC_Test\Projected\2014\Blah.csv')

In [ ]:
cat D:\Annies_Dissertation\MAIAC_Test\Projected\2014\Blah.csv